<a href="https://colab.research.google.com/github/dymiyata/intro-to-ml-and-ai-2025-2026/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression

Here we learn how to perform logistic regression using sklearn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Now we import the dataset we will use.  This is a dataset built in to sklearn.  It has data on a bunch of tumors.  

- We will load all the features into a DataFrame called `X`.  
- The target variable is whether or not the tumor is malignant (cancerous)
  - We have avalue of `1` for malignant
  - We have a value of `0` for not malignant (aka benign).
- We'll load the values of the target variable into `y`

In [6]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

### Split into Training and Testing sets

Now we must split the data into a training set and a testing set.  Let's do a 75/25 train test split.  We'll use 2026 for the `random_state`.

- We'll need to add one more argument to `train_test_split` which is `stratify=y`.  What do you think this does?

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=2026, stratify=y
)

### Preprocessing the Data

Recall, in linear regression it's important for your features to be similar in scale. Why?

The same holds true for logistic regression.  Before I had you just manually and imprecisely scale the features, there is a better way to do things.



To do this, we will take each feature variable $x$ then:
- subtract its mean $\bar{x}$ from each training example so that the mean gets shifted to $0$
- divide each training example by the standard deviation $s$ so that the standard deviation becomes 1.

In this way, you will replace each feature $x$ with $$\frac{x - \bar{x}}{s}.$$  Thus, each resulting feature will have mean 0 and standard deviation 1 so they will all be on the same scale.

Luckily, `sklearn` has a built-in way to do this automatically:

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scaler = StandardScaler()

# learn the mean and st. dev. for each feature from the training set
scaler.fit(X_train)

# scale the features in both the training set and testing set
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Training the Logistic Regression model

Now that we've split the data into training and testing sets and preprocessed the data, we're ready to train the model.  

This part will be almost the same as it was for linear regression:

First we import and define the logistic regression model:

In [19]:
from sklearn.linear_model import LogisticRegression

Now we define the model and fit it to the data.

In [20]:
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

LogisticRegression()

With logistic regression, our model predicts probabilities that $y=1$ meaning probabilities that tumors are malignant.  

We can access these values as follows:

In [21]:
y_train_prob = model.predict_proba(X_train_scaled)
y_test_prob = model.predict_proba(X_test_scaled)

To get the actual predictions (just 0 or 1), we compare the probabilities to a threshold value (by defualt we use a threshould of 0.5).
- values above the threshold are predicted to be 1.
-values below the threshould are predicted to be 0.

`sklearn` does this automatically with just `model.predict()`

In [30]:
y_test_pred = model.predict(X_test_scaled)

### Evaluating the model

Now how do we evaluate the model?

A good way is to check our accuracy on the test set.
- Accuracy just means what percentage of examples we classify correctly.

`sklearn` has a nice function to help us compute this called `accuracy_score`.  Let's import it and use it:

In [28]:
from sklearn.metrics import accuracy_score

In [31]:
accuracy_score(y_test, y_test_pred)

0.972027972027972

Another nice visualization is something called the "Confusion Matrix".  Let's see how that works

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test, y_test_pred)

array([[51,  2],
       [ 2, 88]])

The confusion matrix outputs a $2\times 2$ matrix showing True/False Positives/Negatives.  The order goes:
$$\begin{bmatrix}
\text{TN} & \text{FP} \\
\text{FN} & \text{TP}
\end{bmatrix}$$

I always forget the order and have to look it up...